### Kaggle Fake News Classifier Using Bidirectional LSTM

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('datasets/train/train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df = df.dropna()
df.fillna('')

In [6]:
## get the independent feature
X = df.drop('label',axis = 1)

In [7]:
y= df['label']

In [8]:
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [9]:
X.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
import tensorflow as tf

2025-03-12 12:08:51.480524: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-12 12:08:51.622624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741760631.672610    2180 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741760631.686814    2180 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-12 12:08:51.837260: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [12]:
tf.__version__

'2.18.0'

In [29]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [14]:
### vocabulary size 
voc_size = 5000

### ONEHOT Representation

In [15]:
messages = X.copy()

In [16]:
messages.reset_index(inplace = True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
import nltk
nltk.download('stopwords')

/bin/sh: -c: line 1: syntax error near unexpected token `'stopwords''
/bin/sh: -c: line 1: `nltk.download('stopwords')'


In [19]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [21]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr

[[1179, 2620, 3825, 4770, 3651, 4006, 2148, 519, 4764, 4007],
 [2134, 1702, 3650, 3523, 2025, 1615, 553],
 [1252, 441, 611, 4886],
 [140, 2866, 4374, 1338, 724, 390],
 [2203, 2025, 1616, 2829, 4300, 681, 2025, 3211, 1281, 2593],
 [3755,
  4563,
  2448,
  1947,
  3472,
  331,
  4524,
  1615,
  3723,
  1297,
  3477,
  4079,
  4043,
  240,
  553],
 [4516, 4322, 778, 4259, 2370, 3383, 557, 1727, 4837, 1709, 1794],
 [3687, 534, 1381, 4886, 859, 3302, 331, 3267, 4837, 1709, 1794],
 [2138, 3255, 4729, 3485, 3270, 1724, 793, 4601, 331, 3717],
 [3917, 3034, 3213, 1507, 2798, 112, 2846, 3835],
 [1032, 1959, 1946, 900, 2564, 4658, 724, 1083, 2035, 4917, 362],
 [1338, 4266, 3651, 1724, 331, 859],
 [1227, 3807, 3385, 1050, 25, 1165, 541, 2845, 3766],
 [2445, 1121, 1151, 1287, 135, 381, 3374, 4837, 1709, 1794],
 [3144, 4596, 3933, 3250, 256, 4837, 1709, 1794],
 [2052, 3304, 106, 2103, 1721, 250, 462, 4109, 4838, 107],
 [4235, 814, 1702],
 [2057, 4530, 4706, 1973, 331, 209, 3810, 553],
 [1828, 645, 3

### Embedding Representation

In [22]:
sent_length = 20
embedded_docs =pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  519 4764 4007]
 [   0    0    0 ... 2025 1615  553]
 [   0    0    0 ...  441  611 4886]
 ...
 [   0    0    0 ... 4837 1709 1794]
 [   0    0    0 ... 1627 1169 1326]
 [   0    0    0 ... 3286 2132 1468]]


In [23]:
len(embedded_docs)

18285

In [24]:
## creating model
embedding_vector_features = 40
model = Sequential()


model.add(Embedding(voc_size,embedding_vector_features,input_shape=(sent_length,)))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))


model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/home/harry/Documents/Code/Data_Science/DeepLearning/.venv_tf/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-03-12 12:09:15.376739: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

None


In [41]:
## creating model
embedding_vector_features = 40
model = Sequential()


model.add(Embedding(voc_size,embedding_vector_features,input_shape=(sent_length,)))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation = 'sigmoid'))


model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/home/harry/Documents/Code/Data_Science/DeepLearning/.venv_tf/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 200)            │       112,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 313,001 (1.19 MB)

 Trainable params: 313,001 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

None


In [42]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [43]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [44]:
model.fit(X_train,y_train,validation_data= (X_test,y_test),epochs = 10,batch_size = 64)

Epoch 1/10


192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.7798 - loss: 0.4172 - val_accuracy: 0.9163 - val_loss: 0.1976
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.9462 - loss: 0.1390 - val_accuracy: 0.9234 - val_loss: 0.2029
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9704 - loss: 0.0786 - val_accuracy: 0.9201 - val_loss: 0.2286
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9822 - loss: 0.0518 - val_accuracy: 0.9198 - val_loss: 0.2829
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9914 - loss: 0.0315 - val_accuracy: 0.9153 - val_loss: 0.3408
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9940 - loss: 0.0215 - val_accuracy: 0.9163 - val_loss: 0.4158
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.9961 - loss: 0.0149 - val_accuracy: 0.9104 - val_loss: 0.4989
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.9961 - loss: 0.0142 - val_accuracy: 0.911

In [ ]:
y_pred = model.predict(X_test)
#y_pred[0]

y_pred = [np.argmax(i) for i in y_pred]
y_test[0]
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))


from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step   
[[3419    0]
 [2616    0]]


0.5665285832642917

### Addding Dropout

In [ ]:
from tensorflow.keras.layers import Dropout
### creating 
model.add(Embedding(voc_size,embedding_vector_features,input_shape=(sent_length,)))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

/home/harry/Documents/Code/Data_Science/DeepLearning/.venv_tf/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.fit(X_train,y_train,validation_data= (X_test,y_test),epochs = 10,batch_size = 64)

Epoch 1/10


/home/harry/Documents/Code/Data_Science/DeepLearning/.venv_tf/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['sequential/embedding/embeddings', 'sequential/lstm/lstm_cell/kernel', 'sequential/lstm/lstm_cell/recurrent_kernel', 'sequential/lstm/lstm_cell/bias', 'sequential/dense/kernel', 'sequential/dense/bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5636 - loss: 0.6871 - val_accuracy: 0.5665 - val_loss: 0.6843
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5693 - loss: 0.6839 - val_accuracy: 0.5665 - val_loss: 0.6843
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5579 - loss: 0.6865 - val_accuracy: 0.5665 - val_loss: 0.6844
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5581 - loss: 0.6869 - val_accuracy: 0.5665 - val_loss: 0.6843
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5748 - loss: 0.6819 - val_accuracy: 0.5665 - val_loss: 0.6846
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5603 - loss: 0.6860 - val_accuracy: 0.5665 - val_loss: 0.6847
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5675 - loss: 0.6844 - val_accuracy: 0.5665 - val_loss: 0.6843
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5604 - loss: 0.6862 - val_accuracy: 0.5665 - val

### Performance Metrics and Accuracy

In [59]:
y_pred = model.predict(X_test)
#y_pred[0]
y_pred = (y_pred > 0.5).astype(int)#for binary 

#y_pred = [np.argmax(i) for i in y_pred] #for multiclass
#y_pred = np.array(y_pred)
y_test[0]


189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


np.int64(1)

In [60]:
y_test[1:100]

array([0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0])

In [61]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3117,  302],
       [ 231, 2385]])

In [62]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



In [63]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9116818558409279